In [1]:
from methods import *
from model import LogisticInstance
import numpy as np
import time

In [2]:
# simple 1D case
# e^{x-2} / (1 + e^{x-2}) (-x + 5) --> max
# theoretical answer: x_opt ~ 2.44
# with constraint x<=2 x_opt ~ 2

a = np.array([1])
b = -2
c = np.array([-1])
d = 5
F = np.array([[1]])
g = np.array([2])

issue = LogisticInstance(a, b, c, d, F, g)

In [3]:
def test(method_name, method, real_f_name, real_f, **kwargs):
    t = time.monotonic()
    x_opt = method(**kwargs)
    print('{}:\n  x_opt = {}\n  {} = {}\n  time = {}s'.format(method_name, x_opt,
                                                       real_f_name, real_f(x_opt),
                                                       time.monotonic() - t))

In [5]:
# unconditional optimization

f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)



test('Gradient descent, conv. by argument, h_k = 0.1', 
     GradientDescent(h=0.1), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/k', 
     GradientDescent(h=lambda step: 1. / step), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Gradient descent, conv. by argument, h_k = 1/sqrt(k)', 
     GradientDescent(h=lambda step: step ** -0.5), 
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

print('\n')

test('Accelerated Nesterov gradient descent, h_k = 0.5', 
     AcceleratedNesterovGradientDescent(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

test('Accelerated Nesterov gradient descent, h_k = 1/sqrt(k)', 
     AcceleratedNesterovGradientDescent(h=lambda step: step ** -0.5),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df)

Gradient descent, conv. by argument, h_k = 0.1:
  x_opt = [2.44261703]
  expected_profit = 1.5571455818420734
  time = 0.013306218999787234s
Gradient descent, conv. by argument, h_k = 1/k:
  x_opt = [2.34713717]
  expected_profit = 1.554373820931351
  time = 0.1381585070012079s
Gradient descent, conv. by argument, h_k = 1/sqrt(k):
  x_opt = [2.44255797]
  expected_profit = 1.5571455722441265
  time = 0.00525409799956833s


Accelerated Nesterov gradient descent, h_k = 0.5:
  x_opt = [2.4427758]
  expected_profit = 1.5571455971164652
  time = 0.0014657469982921612s
Accelerated Nesterov gradient descent, h_k = 1/sqrt(k):
  x_opt = [2.44126018]
  expected_profit = 1.5571448252478892
  time = 0.0016612740000709891s


In [9]:
# conditional

# the previous issue (theoretically, x_opt=2)

g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('''Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01''', 
     PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.zeros(issue.a.shape[0]), df=df,
     g=g, dg=dg)

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01:
  x_opt = [2.00233491]
  expected_profit = 1.500582363040146
  time = 0.0697340679980698s


In [10]:
# simple 2d issue
# (x + y) / (1 + e^{x+y}) --> max
# x + y <= 1
#     y <= 0.5
# theoretically, x_opt = (0.5, 0.5)

a = np.array([-1, -1])
b = 0
c = np.array([1, 1])
d = 0
F = np.array([[1, 1],
              [0, 1]])
g = np.array([1, 0.5])

issue = LogisticInstance(a, b, c, d, F, g)

In [11]:
f = lambda x: -issue.log_expected_profit(x)
df = lambda x: -issue.dlog_expected_profit(x)
g = lambda x: np.dot(issue.F , x) -issue.g
dg = lambda x: issue.F

test('''Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01''', 
     PenaltyMethod(),
     'expected_profit', issue.expected_profit,
     f=f, x0=np.ones(issue.a.shape[0]) * 0.01, df=df,
     g=g, dg=dg)

Penalty method, pen_k = [1] * (k-1), ext_pen_func = max(0, x)^0.7,
unconditional method -- gradient descent with h_k=0.01:
  x_opt = [0.50177973 0.50177973]
  expected_profit = 0.26919696224432765
  time = 0.03936207600054331s
